<a href="https://colab.research.google.com/github/Sundaynot/HP_Big_data_project/blob/main/HP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **BIG DATA PROJECT**

This project aims to analyse the seven *Harry Potter* books, written by *J.K.Rowling* between 1997 and 2007.

(1) Initialize some libraries


In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#l'output 'xxx is not a symbolic link' non influirà sull'implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', commenta le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla
#!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
#!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
#!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
#!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
#!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
#!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#Se si verifica un errore relativo all'esecuzione sopra indicata, puoi provare a commentare le 12 righe sopra sotto l'installazione di
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#l'output 'xxx is not a symbolic link' non influirà sulla tua implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', puoi commentare le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla

ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc_proxy.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_5.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbb.so.12': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_0.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind.so.3': File exists
The following additional packages will be installed:
  ca-certificates-java java-common libpcsclite1 libxtst6
  openjdk-8-jre-headless
Suggested packages:
  default-jre pcscd openjdk-8-demo openjdk-8-source libnss-mdns
  fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  ca-certificates-java java-common libpcsclite1 libxtst6
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 up

(2) Create the SparkSession and Clone the repo from personal Github


In [2]:
# Creating the SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("HP_Analysis") \
    .getOrCreate()

print("Session create")

Session create


In [3]:
# Folder's name and repository's URL
repo_name = "HP_Big_data_project"
repo_url = "https://github.com/sundaynot/HP_Big_data_project.git"

# If repo doesn't exist create, else print
if not os.path.exists(repo_name):
    print(f"Cloning repo '{repo_name}'...")
    !git clone {repo_url}
else:
    print(f"Repository '{repo_name}' existing.")

Cloning repo 'HP_Big_data_project'...
Cloning into 'HP_Big_data_project'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 51 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 2.29 MiB | 5.40 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [4]:
# Useful libraries
from pyspark.sql.functions import col, lower, regexp_replace, explode, split, concat_ws, monotonically_increasing_id, lag, when
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from functools import reduce

(3) Read .txt files with Spark

In [5]:
df_hp1 = spark.read.text("/content/HP_Big_data_project/database/01 Harry Potter and the Sorcerers Stone.txt")
df_hp2 = spark.read.text("/content/HP_Big_data_project/database/02 Harry Potter and the Chamber of Secrets.txt")
df_hp3 = spark.read.text("/content/HP_Big_data_project/database/03 Harry Potter and the Prisoner of Azkaban.txt")
df_hp4 = spark.read.text("/content/HP_Big_data_project/database/04 Harry Potter and the Goblet of Fire.txt")
df_hp5 = spark.read.text("/content/HP_Big_data_project/database/05 Harry Potter and the Order of the Phoenix.txt")
df_hp6 = spark.read.text("/content/HP_Big_data_project/database/06 Harry Potter and the Half-Blood Prince.txt")
df_hp7 = spark.read.text("/content/HP_Big_data_project/database/07 Harry Potter and the Deathly Hallows.txt")

(3.1) Show the first row of each file (to see if there are errors)

In [6]:
df_hp1.show(1, truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                 |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mys

In [7]:
df_hp2.show(1, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Not for the first time, an argument had broken out over breakfast at number four, Privet Drive. Mr. Vernon Dursley had been woken in the early hours of the morning by a loud, hooting noise from his nephew Harry’s room.|
+-------------------------------------------------------------------------------------------------------------------

In [8]:
df_hp3.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his h

In [9]:
df_hp4.show(1, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------

In [10]:
df_hp5.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                            

In [11]:
df_hp6.show(1, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                    

In [12]:
df_hp7.show(1, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

(4) Let's process the text

(4.1) Define a function to find the chapters and their relatives names

In [50]:

def process_book_chapters(df_raw, book_number):
    """
    Process a raw DataFrame of a book's text and segments it into chapters,
    recognizing various heading formats ("CHAPTER I", "Chapter 2 - Title", etc.).
    """
    # PHASE 1: Initially cleaning
    # Add to the raw DataFrame a new column "row_id", and mantain the original order of the text (increasing id for each row)
    # Filter removing empty rows and rows composed just by spaces
    # Create a Window that mantains the original order of the text (like the row_id)

    df_ordered = df_raw.withColumn("row_id", F.monotonically_increasing_id())
    df_cleaned = df_ordered.filter((F.col("value").isNotNull()) & (F.trim(F.col("value")) != ""))
    window_spec = Window.orderBy("row_id")

    # PHASE 2: Create the first chapter
    # (for the first chapters there aren't "CHAPTER ONE" or similar, they start with the first row of the book and end when there is "CHAPTER TWO" )
    # So I decide to take the first row for the chapter's name
    first_line_title = df_cleaned.first()["value"]

    # PHASE 3: CHAPTERS' MARKERS
    # A regular expression (RegEx) to find "CHAPTER" or "Chapter"
    # the relative number and the relative chapter's name
    chapter_regex = r"^(?:CHAPTER|Chapter)\s+([A-Za-z0-9]+)(?:\s*[-–:]\s*(.+))?$"

    # add the columns "chapter_match" and "is_new_chapter_line"
    # in "chapter_match" if the row in value matches with chapter_regex
    # insert all the matched string, else an empty string
    # in "is_new_chapter_line" if "chapter_match"!= ""
    # insert TRUE, else insert FALSE
    df_with_markers = df_cleaned.withColumn("chapter_match",
        F.regexp_extract(F.col("value"), chapter_regex, 0)
        ).withColumn("is_new_chapter_line", F.col("chapter_match") != "")

    # add the column "chapter_title_raw"
    # if "is_new_chapter_line"= TRUE
    # in "chapter_title_raw" insert the part of index 2 of the matched string (chapter's name)
    df_with_markers = df_with_markers.withColumn("chapter_title_raw",
        F.when(F.col("is_new_chapter_line"), F.regexp_extract(F.col("value"), chapter_regex, 2)))

    # PHASE 4: CHAPTER'S NAME PROPAGATION
    # If the regex doesn't find the chapter's name, search it in the next row
    # I use lead to take the next row after the specified window
    # add a new column "chapter_title_marker" where:
    # insert lead_value if "is_new_chapter_line"==TRUE and "chapter_title_raw"=""
    # else insert "chapter_title_raw"
    lead_value = F.lead("value").over(window_spec)
    df_with_titles = df_with_markers.withColumn("chapter_title_marker",
        F.when((F.col("is_new_chapter_line")) & (F.col("chapter_title_raw") == ""),
            lead_value).otherwise(F.col("chapter_title_raw")))

    # add the new column "chapter_title_propagated" where
    # fills down the last non-null chapter_name across the window
    # and stores it in a new column called chapter_title.
    df_with_titles = df_with_titles.withColumn("chapter_title_propagated",
        F.last("chapter_title_marker", ignorenulls=True).over(window_spec))

    # PHASE 5: CHAPTER'S ID
    # add a new column chapter_id with the defaul value = 1
    # and everytime is_new_chapter_line=TRUE add 1, else add 0
    df_with_ids = df_with_titles.withColumn("chapter_id",
        F.lit(1) + F.sum(F.when(F.col("is_new_chapter_line"), 1).otherwise(0)).over(window_spec))

    # add the final column "chapter_title", where
    # if "chapter_id"=1 (first chapter) insert the first row for the chapter's name
    # else copy from
    df_with_final_titles = df_with_ids.withColumn("chapter_title",
        F.when(F.col("chapter_id") == 1, F.lit(first_line_title)
        ).otherwise(F.col("chapter_title_propagated")))

    # PHASE 6: Cleaning the text, removing from value CHAPTER and CHAPTER'S NAME
    # add the new column if "is_title_line"
    # where if "is_new_chapter_line"=TRUE insert TRUE in the next row
    # else insert FALSE
    df_with_meta_flags = df_with_final_titles.withColumn( "is_title_line",
        F.lag("is_new_chapter_line", 1, False).over(window_spec) )

    # Filter the text
    # mantain the rows where is_new_chapter_line=FALSE and "is_title_line"=FALSE
    # " ~ " equal to NOT

    df_final_text = df_with_meta_flags.filter(
        (~F.col("is_new_chapter_line")) & (~F.col("is_title_line")) )

    # PHASE 7: GROUPBY
    # Group-by equal chapter_id and equal chapter_title,
    # rename the column "value" with "lines"
    # Create a new column chapter_text by joining all strings in lines with spaces
    # Create a new column "book_id" with the value = book_number from the function intestation
    # select only book_id,chapter_id, chapter_title, and chapter_text and orders rows by chapter_id.
    df_chapters = (
        df_final_text
        .groupBy("chapter_id", "chapter_title")
        .agg(F.collect_list("value").alias("lines"))
        .withColumn("chapter_text", F.concat_ws(" ", F.col("lines")))
        .withColumn("book_id", F.lit(book_number))
        .select("book_id", "chapter_id", "chapter_title", "chapter_text")
        .orderBy("chapter_id"))

    #
    df_chapters = df_chapters.withColumn(
    "chapter_text",
    F.regexp_replace(
        F.trim(F.col("chapter_text")),
        r"^(\w)\s+(.*)$",
        "$1$2"))
    return df_chapters


In [52]:
# Dataframe list
all_hp_dfs = [df_hp1, df_hp2, df_hp3, df_hp4, df_hp5, df_hp6, df_hp7]

processed_books_list = []

print("Starting the elaboration...")

# Use book number to take count of the number of each book
for i, df_book_raw in enumerate(all_hp_dfs):
    book_number = i + 1
    print(f"Processing book {book_number}...")
    df_processed = process_book_chapters(df_book_raw, book_number)
    processed_books_list.append(df_processed)
print("Work done.")

# Only one DataFrame
all_chapters_df = reduce(DataFrame.unionAll, processed_books_list)
all_chapters_df.cache()
print(f"Total chapters: {all_chapters_df.count()}")

# Show the result
all_chapters_df.orderBy("book_id", "chapter_id").show(truncate=80)

Starting the elaboration...
Processing book 1...
Processing book 2...
Processing book 3...
Processing book 4...
Processing book 5...
Processing book 6...
Processing book 7...
Work done.
Total chapters: 195
+-------+----------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|book_id|chapter_id|                                                                   chapter_title|                                                                    chapter_text|
+-------+----------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|      1|         1|Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that th...|Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that th...|
|      1|         2|                                          

In [17]:
# 1. Registra il tuo DataFrame come una "vista" SQL temporanea
all_chapters_df.createOrReplaceTempView("harry_potter_saga")

print("Vista 'harry_potter_saga' creata. Ora puoi usare SQL.")

# 2. Esegui query SQL!

# Quanti capitoli ci sono in ogni libro?
spark.sql("""
    SELECT book_id, COUNT(chapter_id) as num_chapters
    FROM harry_potter_saga
    GROUP BY book_id
    ORDER BY book_id
""").show()

Vista 'harry_potter_saga' creata. Ora puoi usare SQL.
+-------+------------+
|book_id|num_chapters|
+-------+------------+
|      1|          17|
|      2|          17|
|      3|          22|
|      4|          37|
|      5|          38|
|      6|          29|
|      7|          35|
+-------+------------+



DA RIVEDERE


In [18]:
from pyspark.ml.feature import StopWordsRemover # Importa la classe

# 1. Suddividi il testo in parole (tokenizzazione)
# prende la colonna chapter_text, mette tutto in minuscolo e splitta ogni volta che vede uno spazio, e mette tutto nella colonna "words"
df_words = all_chapters_df.select(
    "book_id",
    "chapter_id",
    F.explode(F.split(F.lower(F.col("chapter_text")), r"\s+")).alias("word")
)

# 2. Pulisci le parole (normalizzazione)
# sovrascrive alla colonna word la stessa colonna word di prima rimuovendo la punteggiatura della parole,
# e rimuovendo gli elementi con meno di 2 lettere
df_cleaned_words = df_words.withColumn(
    "word",
    F.regexp_replace(F.col("word"), r"[^\w]", "")
).filter(F.col("word") != "").filter(F.length(F.col("word")) >= 2)

# 3. Aggregi le parole in un array per capitolo
df_word_arrays = df_cleaned_words.groupBy("book_id", "chapter_id").agg(
    F.collect_list("word").alias("words_array"))

# Carica la lista di default per l'inglese
stop_words_list = StopWordsRemover.loadDefaultStopWords("english")

# Aggiungi le tue parole personalizzate
custom_stop_words = stop_words_list + ["harry", "said","ron","weasley","potter","hermione","hagrid","dumbledore","professor","mr"]

# Inizializza il remover SENZA l'argomento 'language'
remover = StopWordsRemover(
    inputCol="words_array",
    outputCol="filtered_words"
)

# IMPOSTA la lista di stop words manualmente
remover.setStopWords(custom_stop_words)

# Applica la trasformazione
df_filtered_arrays = remover.transform(df_word_arrays)

# 5. RIESPLODI l'array filtrato MANTENENDO GLI ID
df_meaningful_words = df_filtered_arrays.select(
    "book_id",        # <-- MANTIENI QUESTO
    "chapter_id",     # <-- MANTIENI QUESTO
    F.explode(F.col("filtered_words")).alias("word")
)

# 6. Esegui il conteggio (Word Count) PER CAPITOLO
df_word_counts_per_chapter = (
    df_meaningful_words.groupBy("book_id", "chapter_id", "word") # <-- MODIFICA QUI
    .count()
    .orderBy(F.col("book_id"), F.col("chapter_id"), F.col("count").desc()) # Ordina per libro, capitolo, e poi per conteggio
)

windowSpec = Window.partitionBy("book_id", "chapter_id").orderBy(F.col("count").desc())

# 2. Assegna un "rank" (una classifica)
# Aggiunge una colonna 'rank' (1, 2, 3...) a ogni parola all'interno del suo capitolo.
df_ranked_words = df_word_counts_per_chapter.withColumn("rank", F.row_number().over(windowSpec))

# 3. Filtra solo per la parola numero 1
# Seleziona solo le righe dove il rank è 1 (la parola più usata).
df_top_word_per_chapter = df_ranked_words.filter(F.col("rank") == 1)

# 4. Mostra il risultato finale
print("La parola più usata in ogni capitolo di ogni libro:")
df_top_word_per_chapter.select("book_id", "chapter_id", "word", "count").orderBy("book_id", "chapter_id").show(200, truncate=False)


La parola più usata in ogni capitolo di ogni libro:
+-------+----------+-----------+-----+
|book_id|chapter_id|word       |count|
+-------+----------+-----------+-----+
|1      |1         |dursley    |45   |
|1      |2         |dudley     |42   |
|1      |3         |uncle      |54   |
|1      |4         |yeh        |35   |
|1      |5         |wand       |27   |
|1      |6         |know       |26   |
|1      |7         |hat        |30   |
|1      |8         |snape      |17   |
|1      |9         |malfoy     |35   |
|1      |10        |troll      |23   |
|1      |11        |snape      |25   |
|1      |12        |back       |25   |
|1      |13        |snape      |33   |
|1      |14        |norbert    |19   |
|1      |15        |back       |20   |
|1      |16        |one        |30   |
|1      |17        |quirrell   |54   |
|2      |1         |dudley     |25   |
|2      |2         |dobby      |58   |
|2      |3         |fred       |35   |
|2      |4         |malfoy     |38   |
|2      |5  

In [19]:
# 1. Definisci la tua lista di incantesimi (minuscoli)
# Concentriamoci su quelli a parola singola per ora
spell_list = [
    "lumos", "nox", "accio", "stupefy", "expelliarmus",
    "riddikulus", "obliviate", "incendio", "protego",
    "sectumsempra", "alohomora", "crucio", "imperio",
    "confringo", "diffindo" ]

multi_word_spells = {
    "avada kedavra": "avada_kedavra",
    "expecto patronum": "expecto_patronum",
    "petrificus totalus": "petrificus_totalus",
    "wingardium leviosa": "wingardium_leviosa"
}

# Definisci la lista completa di token da cercare
all_spell_tokens = spell_list + list(multi_word_spells.values())

# 2. Pre-processa il testo
# Sostituisci "avada kedavra" con "avada_kedavra" PRIMA di splittare
temp_df = all_chapters_df.withColumn("processed_text", F.lower(F.col("chapter_text")))


# (spell = frase da cercare (avada kedavra), token = token con cui sostituirla (avada_kedavra))
for spell, token in multi_word_spells.items():
    temp_df = temp_df.withColumn(
        "processed_text",
        F.regexp_replace(F.col("processed_text"), spell, token)
    )

# 3. Ora tokenizza sul testo "processato"
df_words_adv = temp_df.select(
    "book_id",
    F.explode(F.split(F.col("processed_text"), r"\s+")).alias("word")
)


#Tutti i caratteri che non sono lettere, numeri o underscore
df_cleaned_words_adv = df_words_adv.withColumn(
    "word",
    F.regexp_replace(F.col("word"), r"[^\w_]", "") # Aggiunto "_" per mantenere "avada_kedavra"
)

# 4. Filtra per la nostra lista di token completa
df_spells_adv = df_cleaned_words_adv.filter(
    F.col("word").isin(all_spell_tokens)
)

# 5. Esegui i conteggi
df_total_spell_counts_adv = (
    df_spells_adv.groupBy("word")
    .count()
    .orderBy(F.col("count").desc())
)

print("Conteggio totale (inclusi incantesimi multi-parola):")
df_total_spell_counts_adv.show(truncate=False)

Conteggio totale (inclusi incantesimi multi-parola):
+------------------+-----+
|word              |count|
+------------------+-----+
|expecto_patronum  |36   |
|accio             |33   |
|stupefy           |26   |
|expelliarmus      |25   |
|lumos             |22   |
|avada_kedavra     |19   |
|riddikulus        |16   |
|crucio            |14   |
|petrificus_totalus|11   |
|protego           |11   |
|sectumsempra      |9    |
|imperio           |8    |
|alohomora         |7    |
|wingardium_leviosa|5    |
|diffindo          |5    |
|obliviate         |4    |
|incendio          |3    |
|nox               |2    |
|confringo         |2    |
+------------------+-----+



In [20]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline

# 1. Configura CountVectorizer
# Costruirà un vocabolario delle 10.000 parole più frequenti
# minDF=2.0 significa "ignora le parole che non appaiono in almeno 2 capitoli"
cv = CountVectorizer(
    inputCol="filtered_words",
    outputCol="raw_features",
    vocabSize=10000,
    minDF=2.0
)

# 2. Configura IDF
# Prende le frequenze grezze (raw_features) e calcola il punteggio TF-IDF
idf = IDF(inputCol="raw_features", outputCol="tfidf_features")

# 3. Costruisci la Pipeline
# Una pipeline esegue i passaggi in sequenza.
pipeline = Pipeline(stages=[cv, idf])

# 4. Addestra (fitta) la pipeline sui tuoi dati
# Questo calcola il vocabolario (da cv) e le frequenze dei documenti (da idf)
print("Inizio addestramento pipeline (CV + IDF)...")
pipeline_model = pipeline.fit(df_filtered_arrays)
print("Addestramento completato.")

# 5. Applica il modello per ottenere i vettori TF-IDF
tfidf_df = pipeline_model.transform(df_filtered_arrays)

print("DataFrame con i vettori TF-IDF:")
# Il risultato è un vettore 'sparso'
tfidf_df.select("book_id", "chapter_id", "tfidf_features").show(truncate=80)

Inizio addestramento pipeline (CV + IDF)...
Addestramento completato.
DataFrame con i vettori TF-IDF:
+-------+----------+--------------------------------------------------------------------------------+
|book_id|chapter_id|                                                                  tfidf_features|
+-------+----------+--------------------------------------------------------------------------------+
|      1|        13|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,2...|
|      1|         7|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,2...|
|      1|        11|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,2...|
|      1|        12|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,2...|
|      1|        15|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,2...|
|      1|         3|(10000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,2

In [21]:
from pyspark.sql.types import ArrayType, StructType, StructField, IntegerType, DoubleType

# 6. Estrai il vocabolario dal modello
# Questo è l'elenco di parole. L'indice della parola nell'elenco
# corrisponde all'indice nel vettore sparso.
# pipeline_model.stages[0] è il nostro 'cv' addestrato
vocabulary = pipeline_model.stages[0].vocabulary
vocab_df = spark.createDataFrame(enumerate(vocabulary), ["index", "word"])

# 7. Definisci una UDF (Funzione Definita dall'Utente)
# Questa UDF converte il vettore sparso (es. (10000, [5, 25], [0.1, 0.8]))
# in un array di coppie leggibili: [ (5, 0.1), (25, 0.8) ]
def vector_to_array(v):
    # v è uno SparseVector: (size, indices, values)
    return list(zip([int(i) for i in v.indices], [float(f) for f in v.values]))

# Registra la UDF
to_array_udf = F.udf(vector_to_array,
    ArrayType(StructType([
        StructField("index", IntegerType()),
        StructField("score", DoubleType())
    ]))
)

# 8. Applica la UDF ed "esplodi" i risultati
# Questo crea una riga per ogni parola in ogni capitolo
df_with_scores = tfidf_df.withColumn("scores_array", to_array_udf(F.col("tfidf_features")))

df_exploded = df_with_scores.select(
    "book_id", "chapter_id",
    F.explode(F.col("scores_array")).alias("score_struct")
)

# 9. Unisci (Join) con il vocabolario per riavere le parole
df_word_scores = df_exploded.join(
    vocab_df,
    df_exploded.score_struct.index == vocab_df.index
).select(
    "book_id", "chapter_id", "word", "score_struct.score"
)

# 10. Trova le 5 parole più importanti per ogni capitolo
# Usiamo una Funzione Finestra (Window) per classificare le parole
windowSpec = Window.partitionBy("book_id", "chapter_id").orderBy(F.col("score").desc())

df_top_words = df_word_scores.withColumn("rank", F.row_number().over(windowSpec)) \
                            .filter(F.col("rank") <= 3) \
                            .orderBy("book_id", "chapter_id", "rank")

print("Le 3 parole con punteggio TF-IDF più alto per capitolo:")
df_top_words.show(n=336, truncate=False)

Le 3 parole con punteggio TF-IDF più alto per capitolo:
+-------+----------+------------+------------------+----+
|book_id|chapter_id|word        |score             |rank|
+-------+----------+------------+------------------+----+
|1      |1         |dursley     |112.74866716458313|1   |
|1      |1         |cat         |28.989465254828445|2   |
|1      |1         |drills      |23.350921788663758|3   |
|1      |2         |dudley      |66.74787861489641 |1   |
|1      |2         |piers       |50.593663875438146|2   |
|1      |2         |petunia     |38.53859096151583 |3   |
|1      |3         |vernon      |86.42438175949478 |1   |
|1      |3         |dudley      |68.33711382001299 |2   |
|1      |3         |uncle       |65.7542690289413  |3   |
|1      |4         |yeh         |43.84670389733788 |1   |
|1      |4         |ter         |38.816242111356935|2   |
|1      |4         |yer         |35.03508269228712 |3   |
|1      |5         |griphook    |39.986454122102444|1   |
|1      |5      

Progetto 3: Costruire il Grafo Sociale (PageRank) 🧑‍🤝‍🧑
Come suggerito, puoi vedere chi sono i personaggi "centrali".

Crea una lista di personaggi principali: characters = ["Harry", "Ron", "Hermione", "Dumbledore", "Voldemort", "Snape", "Draco", ...]

Crea gli "archi": Usa all_chapters_df. Per ogni capitolo, se due personaggi (es. "Harry" e "Dumbledore") sono menzionati entrambi nel chapter_text, crea un arco tra loro.

Applica PageRank: Una volta creato il DataFrame degli archi (es. src, dst), puoi usare l'algoritmo PageRank (disponibile in Spark GraphFrames) per trovare il personaggio più importante.

Progetto 4: Similarità tra Capitoli (LSH) 📚
Vuoi trovare capitoli che si assomigliano (es. tutte le partite di Quidditch)? Puoi usare il Locality-Sensitive Hashing (LSH) .

Usa HashingTF e IDF (come nel Progetto 1) per trasformare ogni chapter_text in un vettore di feature.

Applica BucketedRandomProjectionLSH (dalla libreria MLlib) a questi vettori.

L'algoritmo ti restituirà coppie di capitoli che sono "vicini" (simili) con alta probabilità, proprio come descritto nei tuoi appunti .
